# 테스트 키워드 추출

In [ ]:
import pandas as pd
from keybert import KeyBERT
import ast

# ✅ 1. 데이터 불러오기
df = pd.read_csv("C:/Users/hp/Desktop/Bootcamp/PROJECT_OTT_AARRR/video_data_db_with_img.csv").head(100)

# ✅ 2. tags 컬럼: 문자열을 리스트로 변환
df["tags"] = df["tags"].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) and x.startswith("[") else [])

# ✅ 3. title + tags + img_text → 하나의 문장으로 결합
df["combined_text"] = (
    df["title"].fillna("") + " " +
    df["tags"].apply(lambda tags: " ".join(tags)) + " " +
    df["img_text"].fillna("")
)

# ✅ 4. categoryID → 카테고리명 매핑
category_map = {
    1: "엔터테인먼트", 2: "차량", 3: "여행/음식", 4: "게임",
    5: "스포츠", 6: "라이프", 7: "정치", 8: "반려동물",
    9: "교육", 10: "과학/기술"
}
df["category_name"] = df["categoryID"].map(category_map)

# ✅ 5. KeyBERT 모델 로딩
kw_model = KeyBERT(model='paraphrase-multilingual-MiniLM-L12-v2')

# ✅ 6. 키워드 추출 함수 정의
def extract_keywords(text):
    keywords = kw_model.extract_keywords(text, top_n=5)
    return [kw[0] for kw in keywords]

# ✅ 7. 전체 문장에 대해 키워드 추출 (시간 소요될 수 있음)
df["keywords"] = df["combined_text"].apply(extract_keywords)

# ✅ 8. 결과 저장 (선택)
df.to_csv("video_data_with_keywords.csv", index=False)


c:\Users\hp\.conda\envs\YT_project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\hp\.conda\envs\YT_project\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an adminis

In [ ]:
import pandas as pd
from keybert import KeyBERT
import ast
from tqdm import tqdm
import time
import os

# ✅ 파일 경로 설정
INPUT_PATH = "C:/Users/hp/Desktop/Bootcamp/PROJECT_OTT_AARRR/video_data_db_with_img.csv"
OUTPUT_PATH = "C:/Users/hp/Desktop/Bootcamp/PROJECT_OTT_AARRR/video_data_with_keywords.csv"
SAVE_EVERY = 500  # 몇 개마다 저장할지 설정

# ✅ KeyBERT 모델 로딩
kw_model = KeyBERT(model='paraphrase-multilingual-MiniLM-L12-v2')

# ✅ 기존 결과 불러오기 (있으면 이어서)
if os.path.exists(OUTPUT_PATH):
    print("🔁 기존 결과 이어서 불러옵니다...")
    df = pd.read_csv(OUTPUT_PATH)
    df["keywords"] = df["keywords"].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) and isinstance(x, str) and x.startswith("[") else [])
else:
    df = pd.read_csv(INPUT_PATH)
    df["tags"] = df["tags"].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) and x.startswith("[") else [])
    df["combined_text"] = (
        df["title"].fillna("") + " " +
        df["tags"].apply(lambda tags: " ".join(tags)) + " " +
        df["img_text"].fillna("")
    )
    category_map = {
        1: "엔터테인먼트", 2: "차량", 3: "여행/음식", 4: "게임",
        5: "스포츠", 6: "라이프", 7: "정치", 8: "반려동물",
        9: "교육", 10: "과학/기술"
    }
    df["category_name"] = df["categoryID"].map(category_map)
    df["keywords"] = None  # 초기화

# ✅ 키워드 추출 시작
start_time = time.time()
total = len(df)

print(f"📦 총 {total}개 중, 키워드 미추출 항목부터 시작합니다...")

for i in tqdm(range(total), desc="🔍 키워드 추출 중"):
    # 이미 처리된 경우 건너뜀
    if pd.notnull(df.at[i, "keywords"]) and df.at[i, "keywords"] not in [[], "[]", "None", None]:
        continue

    text = df.at[i, "combined_text"]
    keywords = kw_model.extract_keywords(text, top_n=5)
    df.at[i, "keywords"] = [kw[0] for kw in keywords]

    # 중간 저장
    if i % SAVE_EVERY == 0 and i > 0:
        df.to_csv(OUTPUT_PATH, index=False)
        elapsed = time.time() - start_time
        avg = elapsed / (i+1)
        remaining = (total - i - 1) * avg
        print(f"\n💾 중간 저장: {i}/{total} ▶ 경과: {elapsed:.1f}s | 예상 남은 시간: {remaining/60:.1f}분")

# ✅ 최종 저장
df.to_csv(OUTPUT_PATH, index=False)
elapsed = time.time() - start_time
print(f"\n✅ 키워드 추출 완료! 총 소요 시간: {elapsed/60:.2f}분")
print(f"📁 저장 위치: {OUTPUT_PATH}")